In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import warnings
warnings.filterwarnings('ignore')

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
df = pd.read_csv('data_halodoc_unordered.csv', delimiter=';')
df.head()

,name,price,Aturan Pakai,Dosis,Efek Samping,Golongan Produk,Indikasi Umum,Kemasan,Komposisi,Kontra Indikasi,Manufaktur,No. Registrasi,Perhatian,uses
0,Sildenafil Citrate 50 mg 4 Tablet,Rp96.500 - Rp168.500,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet",Sildenafil citrate 50 mg,Hipersensitivitas. Pasien yang menggunakan nit...,Generic Manufacturer,BPOM: GKL1433531717A1* *) Obat ini merupakan o...,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,SILDENAFIL CITRATE merupakan obat yang digunak...
1,Viagra 50 mg 1 Tablet,Rp156.800 - Rp164.900,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Blister @ 4 Tablet Salut Selaput",Sildenafil citrate 50 mg,Pasien yang menggunakan nitrat organik intermi...,Pfizer,BPOM: DKI1690401417A1,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,VIAGRA merupakan obat yang digunakan untuk men...
2,Ericfil Tablet 100 mg,Rp77.100 - Rp104.000,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, muka merah, dispepsia, hidung te...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet salut selaput",Sildenafil citrate 100 mg,"Pasien yang menggunakan nitrat organik, baik s...",Novell Pharmaceutical Laboratories,BPOM: DKL1333528917A1,HARUS DENGAN RESEP DOKTER. Pria yang tidak dia...,ERICFIL merupakan obat yang digunakan untuk me...
3,Cialis Tablet 10 mg,Rp197.300 - Rp203.500,Diberikan sebelum atau setelah makan,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, pusing, dispepsia, nyeri punggun...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 2 Blister @ 2 Tablet Salut Selaput",Tadalafil 10 mg,"Pemberian bersama dengan nitrat oksida, nitrat...",NaN,NaN,NaN,CIALIS 10 MG merupakan obat dengan kandungan T...
4,Ericfil 50 mg Odf,Rp36.500 - Rp44.400,Cara penggunaan Ericfil ODF: 1. Buka kemasan d...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 4 Strip @ 1 ODF",Sildenafil Citrate 50 mg,Hipersensitif terhadap komponen VIASTAR BLUE 5...,Novell Pharmaceutical Laboratories,BPOM: DKL2233554519A1,Hati-hati penggunaan obat in pada pasien denga...,ERICFIL 50 MG ODF merupakan obat yang digunaka...


In [3]:
df.columns

Index(['name', 'price', 'Aturan Pakai', 'Dosis', 'Efek Samping',
       'Golongan Produk', 'Indikasi Umum', 'Kemasan', 'Komposisi',
       'Kontra Indikasi', 'Manufaktur', 'No. Registrasi', 'Perhatian', 'uses'],
      dtype='object')

In [4]:
kolom_teks = [
    'name', 'Aturan Pakai', 'Dosis', 'Efek Samping',
    'Golongan Produk', 'Indikasi Umum', 'Kemasan', 'Komposisi',
    'Kontra Indikasi', 'Manufaktur', 'No. Registrasi', 'Perhatian', 'uses']

df['kombinasi_teks'] = df[kolom_teks].fillna('').agg(' '.join, axis=1)
df['kombinasi_teks'].head()

0    Sildenafil Citrate 50 mg 4 Tablet Dikonsumsi s...
1    Viagra 50 mg 1 Tablet Dikonsumsi satu jam sebe...
2    Ericfil Tablet 100 mg Dikonsumsi satu jam sebe...
3    Cialis Tablet 10 mg Diberikan sebelum atau set...
4    Ericfil 50 mg Odf Cara penggunaan Ericfil ODF:...
Name: kombinasi_teks, dtype: object

## Preprocessing

In [5]:
def preprocessing(teks):
    if pd.isna(teks):
        return ''
    teks = str(teks).lower()
    teks = re.sub(r'[^a-z0-9\s]', '', teks)
    tokens = word_tokenize(teks)
    stop_words = set(stopwords.words('indonesian') + stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

df['prepro_teks'] = df['kombinasi_teks'].apply(preprocessing)
df

,name,price,Aturan Pakai,Dosis,Efek Samping,Golongan Produk,Indikasi Umum,Kemasan,Komposisi,Kontra Indikasi,Manufaktur,No. Registrasi,Perhatian,uses,kombinasi_teks,prepro_teks
0,Sildenafil Citrate 50 mg 4 Tablet,Rp96.500 - Rp168.500,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet",Sildenafil citrate 50 mg,Hipersensitivitas. Pasien yang menggunakan nit...,Generic Manufacturer,BPOM: GKL1433531717A1* *) Obat ini merupakan o...,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,SILDENAFIL CITRATE merupakan obat yang digunak...,Sildenafil Citrate 50 mg 4 Tablet Dikonsumsi s...,sildenafil citrat tablet dikonsumsi jam aktivi...
1,Viagra 50 mg 1 Tablet,Rp156.800 - Rp164.900,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Blister @ 4 Tablet Salut Selaput",Sildenafil citrate 50 mg,Pasien yang menggunakan nitrat organik intermi...,Pfizer,BPOM: DKI1690401417A1,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,VIAGRA merupakan obat yang digunakan untuk men...,Viagra 50 mg 1 Tablet Dikonsumsi satu jam sebe...,viagra tablet dikonsumsi jam aktivita seksual ...
2,Ericfil Tablet 100 mg,Rp77.100 - Rp104.000,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, muka merah, dispepsia, hidung te...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet salut selaput",Sildenafil citrate 100 mg,"Pasien yang menggunakan nitrat organik, baik s...",Novell Pharmaceutical Laboratories,BPOM: DKL1333528917A1,HARUS DENGAN RESEP DOKTER. Pria yang tidak dia...,ERICFIL merupakan obat yang digunakan untuk me...,Ericfil Tablet 100 mg Dikonsumsi satu jam sebe...,ericfil tablet 100 dikonsumsi jam aktivita sek...
3,Cialis Tablet 10 mg,Rp197.300 - Rp203.500,Diberikan sebelum atau setelah makan,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, pusing, dispepsia, nyeri punggun...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 2 Blister @ 2 Tablet Salut Selaput",Tadalafil 10 mg,"Pemberian bersama dengan nitrat oksida, nitrat...",NaN,NaN,NaN,CIALIS 10 MG merupakan obat dengan kandungan T...,Cialis Tablet 10 mg Diberikan sebelum atau set...,ciali tablet makan penggunaan obat sesuai petu...
4,Ericfil 50 mg Odf,Rp36.500 - Rp44.400,Cara penggunaan Ericfil ODF: 1. Buka kemasan d...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 4 Strip @ 1 ODF",Sildenafil Citrate 50 mg,Hipersensitif terhadap komponen VIASTAR BLUE 5...,Novell Pharmaceutical Laboratories,BPOM: DKL2233554519A1,Hati-hati penggunaan obat in pada pasien denga...,ERICFIL 50 MG ODF merupakan obat yang digunaka...,Ericfil 50 mg Odf Cara penggunaan Ericfil ODF:...,ericfil odf penggunaan ericfil odf buka kemasa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,Optimum Nutrition Whey Gold Standard Vanilla 1...,Rp2.200.000,Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,Pangan Olahan Diet Khusus,"Membantu pembentukan otot, mempertahankan mass...",Pack @ 4.54 Kg,"Protein Blend (Whey Protein Isolate, Whey Prot...",NaN,Vita Shopindo,NaN,NaN,Optimum Nutrition On Whey Gold Standard Vanill...,Optimum Nutrition Whey Gold Standard Vanilla 1...,optimum nutrit whey gold standard vanilla diko...
4728,Optimum Nutrition Whey Gold Standard Vanilla 5 Lb,Rp1.150.000,Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,Pangan Olahan Diet Khusus,"Membantu pembentukan o

## Query

In [6]:
corpus = df['prepro_teks'].tolist()
nama_obat = df['name'].tolist()

print(corpus[:5])
print(nama_obat[:5])

['sildenafil citrat tablet dikonsumsi jam aktivita seksual disesuaikan kebutuhan penggunaan obat sesuai petunjuk dokter dewasa sesuai kebutuhan jam aktivita seksual disesuaikan kebutuhan maksim 100 mghari sakit kepala pana kemerahan kulit wajah dispepsia hidung tersumbat gangguan penglihatan diar puse ruam kulit isk obat kera merah informasi obat kalangan medi terapi disfungsi ereksi pria dewasa du strip tablet sildenafil citrat hipersensitivita pasien nitrat organik regul intermiten gener manufactur bpom gkl1433531717a1 obat obat generik nomor registrasi berbeda sesuai ketersediaan stock apotek resep dokter pasien pria dianjurkan aktivita seksual penyakit pasien infark miokard stroke aritmia mengancam nyawa hipertensi td170110 hipotensi td9050 istirahat riwayat gangguan sildenafil citrat obat mengobati gangguan fungsi seksual pria disfungsi ereksi obat meningkatkan aliran darah alat kelamin pria mempertahankan ereksi berhubungan seksual', 'viagra tablet dikonsumsi jam aktivita seksual

In [7]:
query_pendek = ['obat jantung', 'obat stres', 'krim wajah']
query_panjang = [
    'obat oles untuk luka bakar ringan yang tidak perih',
    'obat herbal apa yang sudah terdaftar BPOM',
    'obat flu untuk balita']

query_pendek_prepro = [preprocessing(q) for q in query_pendek]
query_panjang_prepro = [preprocessing(q) for q in query_panjang]

## Information Retrieval

#### 1. Boolean Retrieval

In [8]:
def boolean_retrieval(query, documents, query_original):
    query_terms = query.split()
    results = []

    for i, doc in enumerate(documents):
        doc_terms = set(doc.split())
        query_terms_set = set(query_terms)

        if query_terms_set.issubset(doc_terms):
            results.append({
                'query': query_original,
                'nama_obat': nama_obat[i],
                'matched_terms': list(query_terms_set.intersection(doc_terms)),
                'uses': df['uses'][i]
            })
    return pd.DataFrame(results)

In [9]:
# ir
pd.set_option('display.max_colwidth', None)

for i, query in enumerate(query_pendek_prepro):
    print(f"Query: {query_pendek[i]}")
    results = boolean_retrieval(query, corpus, query_pendek[i])
    display(results.head(10))

for i, query in enumerate(query_panjang_prepro):
    print(f"Query: {query_panjang[i]}")
    results = boolean_retrieval(query, corpus, query_panjang[i])
    display(results.head(10))

Query: obat jantung


,query,nama_obat,matched_terms,uses
0,obat jantung,Ericfil Tablet 100 mg,"[jantung, obat]","ERICFIL merupakan obat yang digunakan untuk mengobati disfungsi ereksi Ericfil mengandung Sildenafil citrate adalah inhibitor selektif dari siklik guanosin monofosfat (cGMP) -spesifik phosphodiesterase tipe 5 (PDE5) Obat ini menghambat fosfodiesterase tipe-5 (PDE-5) yang bertanggung jawab pada proses degradasi siklik guanosin monofosfat (cGMP) di corpus cavernosum Penghambatan PDE-5 meningkatkan cGMP yang menghasilkan relaksasi sel otot polos pembuluh darah paru Sildenafil citrate cepat diserap setelah pemberian oral, dengan bioavailabilitas absolut sekitar 41% Obat ini dimetabolisme di hati (terutama oleh enzim sitokrom P-450 3A4) dan diubah menjadi metabolit aktif dengan sifat yang mirip dengan Sildenafil citrate"
1,obat jantung,Ericfil 50 mg Odf,"[jantung, obat]","ERICFIL 50 MG ODF merupakan obat yang digunakan untuk mengobati gangguan fungsi seksual pada pria (disfungsi ereksi) Obat ini mengandung Sildenafil Citrate, yaitu suatu zat yang termasuk ke dalam golongan PDE-5 Inhibitor yang bekerja dengan cara meningkatkan aliran darah ke alat kelamin pria sehingga dapat mempertahankan ereksi saat berhubungan seksual Ericfil memiliki bentuk sediaan Oral Dissolving Film (ODF) Bentuk sediaan ODF ini, memiliki bentuk yang tipis dan fleksibel, sehingga memudahkan dalam penyimpanan dan nyaman digunakan oleh pasien yang mengalami kesulitan menelan tablet Selain itu, bentuk sediaan ODF mudah untuk menempel pada rongga mulut, cepat terdisolusi (larut) tanpa menggunakan air, dan rapid release"
2,obat jantung,Cialis Tablet 20 mg,"[jantung, obat]","CIALIS 20 MG merupakan obat dengan kandungan Tadalafil Tadalafil digunakan untuk mengobati disfungsi ereksi dan dapat juga digunakan untuk mengobati penderita pembesaran prostat jinak (BPH), hipertensi pulmonal atau tekanan tinggi pada pembuluh darah arteri paru-paru Obat ini bekerja dengan cara melemaskan otot pembuluh darah dan meningkatkan aliran darah ke area tertentu dari tubuh"
3,obat jantung,Ciastar Yellow 10 mg Odf,"[jantung, obat]","CIASTAR YELLOW 10 MG ODF merupakan obat yang digunakan untuk disfungsi ereksi Obat ini mengandung Tadalafil yaitu suatu zat yang termasuk ke dalam golongan PDE-5 Inhibitor yang bekerja dengan cara meningkatkan aliran darah ke alat kelamin pria sehingga dapat mempertahankan ereksi saat berhubungan seksual Ciastar Yellow memiliki bentuk sediaan Oral Dissolving Film (ODF) Bentuk sediaan ODF ini, memiliki bentuk yang tipis dan fleksibel, sehingga memudahkan dalam penyimpanan dan nyaman digunakan oleh pasien yang mengalami kesulitan menelan tablet Selain itu, bentuk sediaan ODF mudah untuk menempel pada rongga mulut, cepat terdisolusi (larut) tanpa menggunakan air, dan rapid release"
4,obat jantung,Viastar Blue 100 mg Odf,"[jantung, obat]","VIASTAR BLUE 100 MG ODFmerupakan obat yang digunakan untuk mengobati gangguan fungsi seksual pada pria (disfungsi ereksi) Obat ini mengandung Sildenafil Citrate, yaitu suatu zat yang termasuk ke dalam golongan PDE-5 Inhibitor yang bekerja dengan cara meningkatkan aliran darah ke alat kelamin pria sehingga dapat mempertahankan ereksi saat berhubungan seksual Viastar Blue memiliki bentuk sediaan Oral Dissolving Film (ODF) Bentuk sediaan ODF ini, memiliki bentuk yang tipis dan fleksibel, sehingga memudahkan dalam penyimpanan dan nyaman digunakan oleh pasien yang mengalami kesulitan menelan tablet Selain itu, bentuk sediaan ODF mudah untuk menempel pada rongga mulut, cepat terdisolusi (larut) tanpa menggunakan air, dan rapid release"
5,obat jantung,Viastar Blue 50 mg Odf,"[jantung, obat]","VIASTAR BLUE 50 MG ODF merupakan obat yang digunakan untuk mengobati gangguan fungsi seksual pada pria (disfungsi ereksi) Obat ini mengandung Sildenafil Citrate, yaitu suatu zat yang termasuk ke dalam golongan PDE-5 Inhibitor yang bekerja dengan cara meningkatkan aliran darah ke alat kelamin pria sehingga dapat mempertahankan ereksi saat berhubungan seksu

Query: obat stres


,query,nama_obat,matched_terms,uses
0,obat stres,Dextamine 10 Tablet,"[obat, stre]","DEXTAMINE merupakan obat dengan kandungan Dexamethasone (glukokortikoid) dan Dexchlorpheniramine Maleate Obat ini bekerja sebagai antiinflamasi, antirematik, serta antialergi/antihistamin"
1,obat stres,Dexteem Plus 10 Tablet,"[obat, stre]","DEXTEEM PLUS TABLET adalah obat kombinasi kortikosteroid Dexamethason dan antihistamin Dexchlorpheniramin Obat ini memiliki efek antiinflamasi, antialergi, dan antihistamin DEXTEEM PLUS TABLET digunakan untuk mengobati kasus-kasus alergi dan inflamasi yang membutuhkan terapi dengan kortikosteroid, seperti gatal-gatal pada kulit, dermatitis (eksim), reaksi alergi obat, rhinitis alergi, asma, dan lain-lain"
2,obat stres,Nutrimax B Complex 30 Tablet,"[obat, stre]","NUTRIMAX B COMPLEX mengandung semua jenis vitamin B dengan komposisi seimbang yang bekerja sinergis memenuhi kebutuhan tubuh, dengan teknologi film coating untuk menghindari kerusakan vitamin B akibat suhu Vitamin B merupakan vitamin larut air yang mudah dikeluarkan dari tubuh, konsumsi setiap hari untuk memenuhi kebutuhan vitamin B Anda Manfaat : - Sebagai nutrisi otak & sistem saraf (Neurotropik) guna membantu mengatasi kelelahan serta stres fisik dan mental - Membantu metabolisme zat gizi untuk menyediakan energi bagi tubuh - Membantu proses pembentukan sel darah merah - Membantu meningkatkan nafsu makan - Membantu menjaga kesehatan kulit, rambut, liver dan jantung"


Query: krim wajah


...

""


Query: obat herbal apa yang sudah terdaftar BPOM


""


Query: obat flu untuk balita


""


#### 2. Jaccard Retrieval

In [10]:
def jaccard(query, documents):
    query_terms = set(query.split())
    results = []

    for i, doc in enumerate(documents):
        doc_terms = set(doc.split())

        intersection = len(query_terms.intersection(doc_terms))
        union = len(query_terms.union(doc_terms))

        if union > 0:
            jaccard_score = intersection / union
            results.append({
                'doc_id': i,
                'nama_obat': nama_obat[i],
                'jaccard_score': jaccard_score,
                'uses': df['uses'][i]
            })
    df_results = pd.DataFrame(results)
    return df_results.sort_values('jaccard_score', ascending=False)

In [11]:
# ir
for i, query in enumerate(query_pendek_prepro):
    print(f"Query: {query_pendek[i]}")
    results = jaccard(query, corpus)
    display(results.head())

for i, query in enumerate(query_panjang_prepro):
    print(f"Query: {query_panjang[i]}")
    results = jaccard(query, corpus)
    display(results.head())

Query: obat jantung


,doc_id,nama_obat,jaccard_score,uses
1997,1997,Konvermex 250 mg 2 Kaplet,0.071429,KONVERMEX 250 MG KAPLET merupakan obat yang digunakan untuk mengatasi infeksi cacing
1998,1998,Konvermex Suspensi 125 mg 10 ml,0.066667,KONVERMEX SUSPENSI 125 MG merupakan obat yang digunakan untuk mengatasi infeksi cacing
1999,1999,Pyrantel 125 mg 4 Kapsul,0.062500,PYRANTEL 125 MG KAPSUL merupakan obat yang digunakan untuk mengatasi infeksi cacing di saluran cerna
39,39,Provula 50 mg 10 Tablet,0.062500,PROVULA 50 MG 10 TABLET merupakan obat yang ditujukan untuk terapi Oligospermia dan Infertilitas anovulatorik
3254,3254,Provula 50 mg 10 Tablet,0.062500,PROVULA 50 MG 10 TABLET merupakan obat yang ditujukan untuk terapi Oligospermia dan Infertilitas anovulatorik


Query: obat stres


,doc_id,nama_obat,jaccard_score,uses
1997,1997,Konvermex 250 mg 2 Kaplet,0.071429,KONVERMEX 250 MG KAPLET merupakan obat yang digunakan untuk mengatasi infeksi cacing
1998,1998,Konvermex Suspensi 125 mg 10 ml,0.066667,KONVERMEX SUSPENSI 125 MG merupakan obat yang digunakan untuk mengatasi infeksi cacing
3254,3254,Provula 50 mg 10 Tablet,0.062500,PROVULA 50 MG 10 TABLET merupakan obat yang ditujukan untuk terapi Oligospermia dan Infertilitas anovulatorik
39,39,Provula 50 mg 10 Tablet,0.062500,PROVULA 50 MG 10 TABLET merupakan obat yang ditujukan untuk terapi Oligospermia dan Infertilitas anovulatorik
1999,1999,Pyrantel 125 mg 4 Kapsul,0.062500,PYRANTEL 125 MG KAPSUL merupakan obat yang digunakan untuk mengatasi infeksi cacing di saluran cerna


Query: krim wajah


...

,doc_id,nama_obat,jaccard_score,uses
3224,3224,Obat Gosok Bunga Merah 40 ml,0.090909,Obat Gosok Bunga Merah merupakan minyak gosok yang membantu meredakan luka dan memar akibat gigitan serangga
2603,2603,Lanakeloid-E Cream 10 g,0.088889,"LANAKELOID-E CREAM merupakan obat dengan kandungan Centella Asiatica Phytosome, Vit E dalma bentuk krim Obat ini digunakan untuk membantu proses penyembuhan luka bakar ringan"
2632,2632,Lanakeloid 10 Kaplet,0.085106,LANAKELOID merupakan obat dengan kandungan Centella Asiatica dalam bentuk kaplet Obat ini digunakan untuk membantu meringankan luka bakar ringan
2375,2375,Aza 20 gel 30 g,0.083333,AZA 20 GEL Pengobatan akne vulgaris ringan sampai sedang
1562,1562,Pamol Sirup 60 ml,0.076923,"PAMOL SIRUP merupakan obat yang di gunakan sebagai penurun demam dan pereda nyeri seperti sakit kepala , sakit gigi dan nyeri ringan Obat ini mengandung paracetamol yang memiliki fungsi sebagai antipiretik dan analgesik"


Query: obat herbal apa yang sudah terdaftar BPOM


,doc_id,nama_obat,jaccard_score,uses
3224,3224,Obat Gosok Bunga Merah 40 ml,0.157895,Obat Gosok Bunga Merah merupakan minyak gosok yang membantu meredakan luka dan memar akibat gigitan serangga
36,36,Himalaya Shatavari 60 Kapsul,0.125000,HIMALAYA SHATAVARI 60 KAPSUL merupakan obat herbal yang berguna untuk memperbaiki level hormon estrogen dan siklus haid
1622,1622,Bodrex Herbal Batuk 15 ml 5 Sachet,0.115385,Bodrex Herbal Batuk merupakan obat tradisional yang di formulasikan dari sari tumbuh-tumbuhan berkhasiat Bodrex Herbal Batuk ini digunakan untuk membantu meredakan batuk dan melegakakan tenggorokan
1261,1261,Bodrex Herbal Batuk 15 ml 5 Sachet,0.115385,Bodrex Herbal Batuk merupakan obat tradisional yang di formulasikan dari sari tumbuh-tumbuhan berkhasiat Bodrex Herbal Batuk ini digunakan untuk membantu meredakan batuk dan melegakakan tenggorokan
4586,4586,Redacid 250 mg 6 Kaplet,0.100000,REDACID 250 MG TABLET merupakan obat herbal untuk meringankan gangguan pada lambung


Query: obat flu untuk balita


,doc_id,nama_obat,jaccard_score,uses
995,995,Bodrex Flu Dan Batuk Berdahak Pe Sirup 60 ml,0.068966,"BODREX FLU DAN BATUK BERDAHAK PE SIRUP mengandung Paracetamol, Bromhexine HCL, Pseudoephedrine HCl, dan Glyceryl Guaiacolate Obat ini digunakan untuk mengatasi gejala flu seperti demam, sakit kapala, bersin-bersin dan hidung tersumbat yang disertai batuk berdahak"
1997,1997,Konvermex 250 mg 2 Kaplet,0.066667,KONVERMEX 250 MG KAPLET merupakan obat yang digunakan untuk mengatasi infeksi cacing
1001,1001,Decolgen Kids Flu Sirup 60 ml,0.064516,"DECOLGEN KIDS FLU SIRUP 60 ML merupakan obat dengan kandungan Paracetamol, Pseudoephedrine HCl, Chlorpheniramine maleate Obat ini dapat digunakan untuk meredakan gejala flu seperti demam, sakit kepala, bersin, dan hidung tersumbat"
921,921,Decolsin 4 Kapsul,0.062500,"DECOLSIN merupakan obat dengan kandungan Paracetamol, Pseudoephedrine HCl, Chlorpheniramine maleate, Dextromethorphan HBr, Guaifenesin Obat ini dapat digunakan untuk meredakan batuk yang disertai gejala flu seperti demam, sakit kepala, hidung tersumbat, dan bersin"
1998,1998,Konvermex Suspensi 125 mg 10 ml,0.062500,KONVERMEX SUSPENSI 125 MG merupakan obat yang digunakan untuk mengatasi infeksi cacing


#### 3. TF-IDF

In [12]:
tfidf_vectorizer = TfidfVectorizer(
    max_features = 5000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2)
)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

def tfidf_retrieval(query, documents, vectorizer, matrix):
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, matrix).flatten()
    results = []
    for i, sim in enumerate(similarities):
        if sim > 0:
            results.append({
                'doc_id': i,
                'nama_obat': nama_obat[i],
                'tfidf_score': sim,
                'uses': df['uses'][i]
            })
    results.sort(key=lambda x: x['tfidf_score'], reverse=True)
    return pd.DataFrame(results)

In [13]:
# ir 
for i, query in enumerate(query_pendek_prepro):
    print(f"Query: {query_pendek[i]}")
    results = tfidf_retrieval(query, corpus, tfidf_vectorizer, tfidf_matrix)
    display(results.head(10))
for i, query in enumerate(query_panjang_prepro):
    print(f"Query: {query_panjang[i]}")
    results = tfidf_retrieval(query, corpus, tfidf_vectorizer, tfidf_matrix)
    display(results.head(10))

Query: obat jantung


,doc_id,nama_obat,tfidf_score,uses
0,480,Combivent UDV 10 Vial,0.336614,COMBIVENT UDV merupakan obat asma yang mengandung Ipratropium bromide dan Salbutamol sulphate Salbutamol dan Ipratropium memiliki fungsi sebagai bronkodilator Obat ini digunakan untuk pengobatan COPD (Chronic Obstructive Pulmonary Disease)
1,494,Innovair Inhalation Solution,0.271684,"INNOVAIR INHALATION SOLUTION merupakan obat yang mengandung Beclomethasone dipropionate dan formoterol fumarate dihydrate Beclomethasone dipropionate merupakan salah satu obat golongan kortikosteroid yang bekerja sebagai anti radang yang akan mengurangi radang arau pembengkakakan pada kantung-kantung udara yang terletak di ujung saluran udara terkecil (bronkiolus) di dalam paru-paru Sedangkan, Formoterol fumarate dihydrate bekerja sebagai bronkodilator, yaitu obat yang akan memperlebar saluran nafas, sehingga proses bernafas dan pertukaran antara oksigen dan karbondioksida menjadi lebih mudah INNOVAIR inhaler digunakan untuk meringankan gejala mengi dan sesak nafas sebagai terapi pada pada penderita asma"
2,947,Anadex 10 Kaplet,0.242251,"ANADEX TABLET mengandung zat aktif Paracetamol, Phenylpropanolamine HCl, Chlorpheniramine Maleat, dan Dextromethorphan HBr Obat ini digunakan untuk mengatasi gejala flu seperti demam, sakit kepala, bersin-bersin dan hidung tersumbat yang disertai batuk Obat ini menyebabkan kantuk Jangan mengendarai kendaraan bermotor atau menjalankan mesin ketika menggunakan obat ini"
3,1124,Trifed 10 Tablet,0.224411,"TRIFED merupakan obat yang digunakan untuk mengobati gejala flu (hidung gatal, hidung tersumbat dan bersin-bersin), batuk pilek (selesma) dan allergic rhinitis Trifed mengandung Pseudoephedrine dan Triprolidine Pseudoephedrine adalah zat dekongestan (simpatomimetik) Pseudoephedrine bekerja dengan mengecilkan pembuluh darah untuk mengurangi pembengkakan dan penyumbatan Triprolidine bekerja dengan cara menghambat efek dari zat histamin pada sel-sel tubuh"
4,1469,Lanareuma 20 mg 10 Kapsul,0.221907,"Lanareuma adalah produk obat dengan zat aktif Piroxicam yang memiliki sifat analgesik (meredakan nyeri), anti-inflamasi (meredakan peradangan) dan antipiretik (menurunkan demam) Lanareuma bekerja dengan cara menghambat reversibel enzim siklooksigenase-1 dan -2 (COX-1 dan -2), sehingga mengakibatkan penurunan pembentukan prekursor prostaglandin"
5,1419,Pronalges Suppositoria 100 mg,0.215730,"PRONALGES SUPPOSITORIA 100 MG mengandung zat aktif Ketoprofen, jenis obat anti inflamasi non steroid (OAINS) Obat ini digunakan untuk mengatasi nyeri ringan hingga sedang pada penyakit arthritis rheumatoid, osteoarthritis, asam urat (gout), ankylosing spondilitis, migrain, sakit kepala, sakit telinga, nyeri pasca operasi, nyeri haid (dismenore) serta nyeri akibat trauma otot Obat ini digunakan dengan cara dimasukkan ke dalam anus"
6,1847,Scopma 10 mg 10 Kaplet,0.211518,"SCOPMA 10 MG adalah obat yang di gunakan untuk mengobati nyeri perut atau pencernaan yang tergolong dalam obat keras Scopma mengandung Hyoscine butylbromide memiliki efek spasmolitik pada otot polos traktus gastrointestinal, biliaris, urinarius, dan uterus Obat ini umum digunakan untuk mengobati kram perut, kolik ginjal, dan kejang kandung kemih Obat ini mempunyai efek samping yang umumnya seperti kantuk, memicu glaukoma, dan reaksi alergi"
7,4250,Nu Q Ten Com 10 Kapsul,0.194983,NU Q TEN COM 10 KAPSUL merupakan suplemen yang mengandung Q10 Q10 digunakan sebagai katalis alami untuk pembentukan energi dari makanan dan sebagai antioksidan untuk mencegah kerusakan sel Memelihara kesehatan jantung
8,922,Dicom 30/2.5 mg 10 Tablet,0.191604,"Dicom 30/2 5 mg Tablet adalah obat yang digunakan untuk mengatasi hidung tersumbat karena alergi, flu, dan peradangan pada dinding sinus hidung (sinusitis) Obat ini mengandung zat aktif pseudoephedrine HCl dan triprolidine HCl Pseudoephedrine bekerja dengan mempersempit pembuluh darah untuk mengurangi pembengkakan pada hidung Sementara, triprolidine HCl bekerja dengan mengh

Query: obat stres


,doc_id,nama_obat,tfidf_score,uses
0,2579,Ottogenta 0.1% Cream 5 g,0.348855,OTTOGENTA KRIM merupakan obat dengan kandungan Gentamicin sulphate dalam bentuk krim Obat ini digunakan untuk infeksi kulit ringan primer maupun sekunder
1,2533,Sulfadiazine Silver Cream 35 g,0.307485,Silver sulfadiazine merupakan obat yang digunakan untuk merawat dan memulihkan infeksi luka dari penggunanya dan luka bakar tubuh
2,2699,Cotrimoxazole 480 mg 10 Tablet,0.306690,"COTRIMOXAZOLE 480 MG TABLET adalah tablet antibiotik kombinasi trimethoprim 80 mg dan sulfamethoxazole 400 mg Obat ini digunakan untuk menangani infeksi yang disebabkan oleh bakteri, seperti bronkitis, otitis media, dan infeksi saluran kemih Selain itu, kotrimoksazol juga dapat digunakan untuk menangani dan mencegah Pneumocystis Carinii Pneumonia (PCP) pada pasien dengan daya tahan tubuh turun, seperti penderita HIV/AIDS Obat ini bekerja dengan menghambat pembentukan DNA dan protein bakteri"
3,2688,Isoniazid 300 mg 10 Tablet,0.298176,"ISONIAZID merupakan obat golongan antibiotik yang digunakan untuk menangani gejala tuberkulosis paru ataupun ekstra paru, dimana biasanya regimen Isoniazid termasuk kedalam regimen penanganan Tuberkulosis (TBC) yang terdiri dari Rifampisin, Izoniazid, Pirazinamid, dan Ethambutol Obat ini bekerja dengan dengan menghambat asam mikolat yang merupakan komponen penting dalam sel bakteri"
4,2454,Pyderma 0.25% Cream 10 g,0.293501,PYDERMA merupakan obat krim yang mengandung zat aktif Desoxymethasone Desoxymethasone termasuk obat golongan kortikosteroid yang memiliki efek antiinflamasi Obat ini digunakan untuk meredakan radang dan pruritik (rasa gatal) pada dermatitis yang responsif terhadap kortikosteroid Desoxymethasone bekerja dengan cara mengaktifkan reseptor intraselular yang spesifik Pengaktifan ini menginduksi sintesis dari protein anti-inflamasi dan penekanan sintests mediator inflamasi Proses ini menyebabkan penurunan respon inflamasi dan respon autoimun
5,948,Crofed 10 Tablet,0.291093,"CROFED merupakan obat yang di gunakan untuk mengobati gejala flu dan allergic rhinitis Crofed mengandung tripolidine dan pesudoefedrin Tripolidine adalah obat golongan antihistamin yang memiliki sifat antikolinergik Obat ini digunakan untuk mengurangi gejala yang terkait dengan alergi Obat ini sering dikombinasikan dengan obat lain untuk mengurangi gejala flu Pesudoefedrin adalah obat yang digunakan sebagai nasal dekongestan, stimulan, dan sebagai wakefulness promoting agent Obat ini termasuk obat simpatomimetik dari kelas phenethylamine dan amfetamin Obat ini biasanya digunakan dalam bentuk garamnya yaitu pseudoephedrine hydrochloride"
6,1584,Paracetamol Enema 125 mg/2.5 ml,0.289275,"Paracetamol Enema adalah obat yang di gunakan sebagai penurun demam dan pereda nyeri seperti sakit kepala, sakit gigi dan nyeri ringan Obat ini dikemas dalam bentuk enema yaitu prosedur pemasukan cairan obat ke dalam kolon melalui anus, untuk anak atau pasien yang kesulitan untuk menelan obat Paracetamol bekerja pada pusat pengatur suhu di hipotalamus untuk menurunkan suhu tubuh (antipiretik) serta menghambat sintesis prostaglandin sehingga dapat mengurangi nyeri ringan sampai sedang (analgesik)"
7,2644,FG Troches 10 Tablet,0.288710,"FG TROCHES merupakan tablet hisap Antibiotik yang memiliki kandungan Fradiomycin Sulfate dan Gramicidin-S HCl yang berfungsi untuk mengobati gingivitis, stomatitis, faringitis, bronkitis, tonsilitis, angina vincent, difteri faringeal, priodontitis"
8,2710,Ethambutol 500 mg 10 Tablet,0.286783,"ETHAMBUTOL 500 MG TABLET adalah obat antibiotik yang digunakan untuk mengobati tuberkulosis (TBC), terutama bila diduga telah terjadi resistensi Obat ini biasanya digunakan secara kombinasi dengan obat TBC lainnya, seperti isoniazid, rifampicin, dan pyrazinamide"
9,39,Provula 50 mg 10 Tablet,0.274422,PROVULA 50 MG 10 TABLET merupakan obat yang ditujukan untuk terapi Oligospermia dan Infertilitas anovulatorik


Query: krim wajah


...

,doc_id,nama_obat,tfidf_score,uses
0,2616,Mebo Salep 20 gr,0.495767,"MEBO SALEP 20 GR merupakan salah satu sediaan salep herbal luka bakar yang diformulasikan dengan bahan-bahan herbal alami seperti : Ekstrak Akar mangkokan, ekstrak kulit chinensis Phellodendri, Ekstrak rimpang Coptidis"
1,2570,Mertus Cream 10 gr,0.390812,"Mertus adalah antibiotik topikal (untuk pemakaian luar) yang digunakan untuk mengobati infeksi primer dan sekunder, seperti impetigo (infeksi kulit yang berupa lepuh atau bercak luka terbuka pada kulit), luka bakar, luka pasca operasi, dan luka terinfeksi Mertus mengandung zat aktif Mupirocin calcium bersifat bakteriostatik (menghambat pertumbuhan bakteri) dan bakterisidal (membunuh bakteri) pada konsentrasi yang lebih tinggi"
2,2632,Lanakeloid 10 Kaplet,0.390477,LANAKELOID merupakan obat dengan kandungan Centella Asiatica dalam bentuk kaplet Obat ini digunakan untuk membantu meringankan luka bakar ringan
3,2603,Lanakeloid-E Cream 10 g,0.384690,"LANAKELOID-E CREAM merupakan obat dengan kandungan Centella Asiatica Phytosome, Vit E dalma bentuk krim Obat ini digunakan untuk membantu proses penyembuhan luka bakar ringan"
4,2468,Bioplacenton Gel 15 g,0.372769,"BIOPLACENTON merupakan obat yang mengandung Placenta Extract dan Neomycin sulfate Obat ini biasa digunakan untuk mengobati luka bakar, luka dengan infeksi, serta luka kronik dan jenis luka yang lain Ekstrak plasenta bekerja dengan memicu pembentukan jaringan baru dan untuk wound healing, sedangkan neomycin bekerja dengan mencegah atau mengatasi infeksi bakteri Gram negatif pada area luka"
5,3568,Vaseline Repairing Petroleum Jelly 100 ml,0.367285,"VASELINE REPAIRING PETROLEUM JELLY merupakan produk perawatan kulit dengan kandungan White Petrolatum Produk ini digunakan untuk mengurangi tampaknya garis halus dan kering, membantu menyembuhkan luka gores kecil, kulit mengelupas, dan luka bakar, mencegah kulit kasar akibat pecah-pecah"
6,2480,Burnazin 10 mg/g Cream 35 g,0.348693,BURNAZIN CREAM adalah obat yang mengandung Silver Sulfadiazine Obat ini digunakan untuk mengobati atau mencegah infeksi pada luka bakar yang rentan terhadap infeksi sekunder
7,2596,Dermatix Ultra Gel 5 g,0.302759,DERMATIX ULTRA merupakan produk perawatan bekas luka yang mengandung Vitamin C Ester yang teruji secara klinis mampu pudarkan bekas luka hingga 80%
8,2540,Centabio Gel 20 g,0.298863,"CENTABIO GEL 20 G adalah obat yang digunakan untuk membantu penyembuhan luka bakar, borok kronis, dan cacat kulit yang disertai infeksi bakteri tertentu Centabio mengandung neomycin yang bekerja dengan cara mengikat secara reversibel terhadap sub unit 30s dari ribosom bakteri sehingga menghambat sintesa protein yang pada akhirnya menghambat pertumbuhan bakteri itu"
9,2631,Laderma Cream 8 ml,0.285166,"Laderma Cream 8 ml membantu menghidrasi, melembabkan dan melindungi kulit kering Laderma dapat diberikan kepada anak-anak dan dewasa"


Query: obat herbal apa yang sudah terdaftar BPOM


,doc_id,nama_obat,tfidf_score,uses
0,1235,Komix Herbal 15 ml 4 Botol,0.299332,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
1,1596,Komix Herbal 15 ml 4 Botol,0.299332,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
2,1246,Komix Herbal Lemon Sirup 15 ml 4 Botol,0.297617,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
3,1607,Komix Herbal Lemon Sirup 15 ml 4 Botol,0.297617,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
4,1277,Komix Herbal Rasa Jeruk Nipis 15 ml 6 Sachet,0.292897,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
5,1638,Komix Herbal Rasa Jeruk Nipis 15 ml 6 Sachet,0.292897,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
6,1247,Komix Herbal Original 15 ml 6 Sachet,0.292466,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
7,1608,Komix Herbal Original 15 ml 6 Sachet,0.292466,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
8,1278,Komix Herbal Rasa Peppermint 15 ml 6 Sachet,0.290732,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
9,1639,Komix Herbal Rasa Peppermint 15 ml 6 Sachet,0.290732,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"


Query: obat flu untuk balita


,doc_id,nama_obat,tfidf_score,uses
0,1001,Decolgen Kids Flu Sirup 60 ml,0.339947,"DECOLGEN KIDS FLU SIRUP 60 ML merupakan obat dengan kandungan Paracetamol, Pseudoephedrine HCl, Chlorpheniramine maleate Obat ini dapat digunakan untuk meredakan gejala flu seperti demam, sakit kepala, bersin, dan hidung tersumbat"
1,948,Crofed 10 Tablet,0.309990,"CROFED merupakan obat yang di gunakan untuk mengobati gejala flu dan allergic rhinitis Crofed mengandung tripolidine dan pesudoefedrin Tripolidine adalah obat golongan antihistamin yang memiliki sifat antikolinergik Obat ini digunakan untuk mengurangi gejala yang terkait dengan alergi Obat ini sering dikombinasikan dengan obat lain untuk mengurangi gejala flu Pesudoefedrin adalah obat yang digunakan sebagai nasal dekongestan, stimulan, dan sebagai wakefulness promoting agent Obat ini termasuk obat simpatomimetik dari kelas phenethylamine dan amfetamin Obat ini biasanya digunakan dalam bentuk garamnya yaitu pseudoephedrine hydrochloride"
2,1060,Bodrex Flu Dan Batuk Tidak Berdahak Pe Sirup 60 ml,0.307038,"BODREX FLU DAN BATUK TIDAK BERDAHAK PE SIRUP merupakan obat yang mengandung Paracetamol, Phenylephrine, dan Dextrometorphan HBr Obat ini digunakan untuk mengatasi gejala flu seperti demam, sakit kepala, hidung tersumbat, dan bersin-bersin yang disertai batuk tidak berdahak"
3,995,Bodrex Flu Dan Batuk Berdahak Pe Sirup 60 ml,0.296608,"BODREX FLU DAN BATUK BERDAHAK PE SIRUP mengandung Paracetamol, Bromhexine HCL, Pseudoephedrine HCl, dan Glyceryl Guaiacolate Obat ini digunakan untuk mengatasi gejala flu seperti demam, sakit kapala, bersin-bersin dan hidung tersumbat yang disertai batuk berdahak"
4,938,Bodrex Flu Dan Batuk Berdahak Pe 4 Kaplet,0.287588,"BODREX FLU DAN BATUK BERDAHAK PE mengandung Paracetamol, Phenylephrine HCl, Glyceryl Guaiacolate, Bromhexine HCl Komponen tersebut bekerja sebagai analgesik-antipiretik, dekongestan hidung, ekspektoran dan mukolitik Obat ini digunakan untuk mengatasi gejala flu seperti demam, sakit kepala, bersin-bersin dan hidung tersumbat yang disertai batuk berdahak"
5,1029,OBH Combi Batuk Plus Flu Jahe 100 ml,0.285709,"OBH COMBI BATUK PLUS FLU adalah obat yang digunakan untuk meredakan batuk yang disertai gejala flu seperti demam, sakit kepala, bersin-bersin dan hidung tersumbat"
6,953,Mixagrip Flu 4 Kaplet,0.271454,"MIXAGRIP FLU adalah obat yang digunakan untuk meringankan gejala-gejala flu seperti demam, sakit kepala, hidung tersumbat dan bersin-bersin Mixagrip flu mengandung Paracetamol, Phenylephrine HCl dan Chlorphenamine Maleate yang bekerja sebagai analgetik-antipiretik, antihistamin dan dekongestan hidung"
7,956,OBH Combi Plus Batuk Flu Menthol 60 ml,0.262598,"OBH COMBI PLUS BATUK FLU adalah obat yang digunakan untuk meredakan batuk yang disertai gejala flu seperti deman, sakit kepala, bersin dan hidung tersumbat Kandungan dalam OBH Combi Batuk Plus Flu bekerja sebagai ekspektoran (membantu mengeluarkan dahak), antihistamin (mengurangi gejala alergi), analgesik-antipiretik (menurunkan panas, demam dan meredakan sakit kepala) dan dekongestan hidung (melonggarkan saluran pernapasan)"
8,1047,Siladex Flu 60 ml,0.256415,"SILADEX FLU mengandung zat aktif paracetamol, guaifenesin, bromheksin hcl, chlorpeniramine maleat, dan pseudoefedrin HCl Obat ini berfungsi untuk meringankan gejala - gejala flu seperti demam, sakit kepala, hidung tersumbat dan bersin - bersin serta sebagai mukolitik-ekspektoran (mengencerkan dahak sehingga mudah dikeluarkan) dalam meredakan batuk berdahak"
9,1030,OBH Combi Batuk Plus Flu Jahe 60 ml,0.249329,"OBH COMBI PLUS BATUK FLU JAHE adalah obat batuk hitam dengan rasa jahe Obat ini mengandung Paracetamol, yang bekerja sebagai pereda demam (antipiretik) sekaligus mengurangi rasa nyeri di kepala (analgetik) Kandungan Ekstrak Succus liquiritiae dan Ammonium chloride bekerja sebagai pereda batuk berdahak (ekspektoran), Ephedrine sebagai dekongestan yang bertindak melonggarkan saluran pernafasan sehingga mencegah hidung tersumbat kar

#### 4. Cosine Similarity Retrieval

In [14]:
bow_vectorizer = CountVectorizer(
    max_features = 5000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1,2)
)
bow_matrix = bow_vectorizer.fit_transform(corpus)

from sklearn.metrics.pairwise import cosine_similarity as sklearn_cosine_similarity

def cosine(query, documents, vectorizer, matrix):
    query_vector = vectorizer.transform([query])
    similarities = sklearn_cosine_similarity(query_vector, matrix).flatten()
    results = []
    for i, sim in enumerate(similarities):
        if sim > 0:
            results.append({
                'doc_id': i,
                'nama_obat': nama_obat[i],
                'cosine_score': sim,
                'uses': df['uses'][i]
            })
    results.sort(key=lambda x: x['cosine_score'], reverse=True)
    return pd.DataFrame(results)    

In [15]:
# ir
for i, query in enumerate(query_pendek_prepro):
    print(f"Query: {query_pendek[i]}")
    results = cosine(query, corpus, bow_vectorizer, bow_matrix)
    display(results.head(10))
for i, query in enumerate(query_panjang_prepro):
    print(f"Query: {query_panjang[i]}")
    results = cosine(query, corpus, bow_vectorizer, bow_matrix)
    display(results.head(10))

Query: obat jantung


,doc_id,nama_obat,cosine_score,uses
0,480,Combivent UDV 10 Vial,0.511478,COMBIVENT UDV merupakan obat asma yang mengandung Ipratropium bromide dan Salbutamol sulphate Salbutamol dan Ipratropium memiliki fungsi sebagai bronkodilator Obat ini digunakan untuk pengobatan COPD (Chronic Obstructive Pulmonary Disease)
1,494,Innovair Inhalation Solution,0.479022,"INNOVAIR INHALATION SOLUTION merupakan obat yang mengandung Beclomethasone dipropionate dan formoterol fumarate dihydrate Beclomethasone dipropionate merupakan salah satu obat golongan kortikosteroid yang bekerja sebagai anti radang yang akan mengurangi radang arau pembengkakakan pada kantung-kantung udara yang terletak di ujung saluran udara terkecil (bronkiolus) di dalam paru-paru Sedangkan, Formoterol fumarate dihydrate bekerja sebagai bronkodilator, yaitu obat yang akan memperlebar saluran nafas, sehingga proses bernafas dan pertukaran antara oksigen dan karbondioksida menjadi lebih mudah INNOVAIR inhaler digunakan untuk meringankan gejala mengi dan sesak nafas sebagai terapi pada pada penderita asma"
2,948,Crofed 10 Tablet,0.463184,"CROFED merupakan obat yang di gunakan untuk mengobati gejala flu dan allergic rhinitis Crofed mengandung tripolidine dan pesudoefedrin Tripolidine adalah obat golongan antihistamin yang memiliki sifat antikolinergik Obat ini digunakan untuk mengurangi gejala yang terkait dengan alergi Obat ini sering dikombinasikan dengan obat lain untuk mengurangi gejala flu Pesudoefedrin adalah obat yang digunakan sebagai nasal dekongestan, stimulan, dan sebagai wakefulness promoting agent Obat ini termasuk obat simpatomimetik dari kelas phenethylamine dan amfetamin Obat ini biasanya digunakan dalam bentuk garamnya yaitu pseudoephedrine hydrochloride"
3,1020,Longatin 25 mg 4 Kapsul,0.453743,LONGATIN 25 MG KAPSUL merupakan obat yang mengandung zat aktif Noscapin Obat ini digunakan untuk membantu mengatasi batuk kering
4,1847,Scopma 10 mg 10 Kaplet,0.444649,"SCOPMA 10 MG adalah obat yang di gunakan untuk mengobati nyeri perut atau pencernaan yang tergolong dalam obat keras Scopma mengandung Hyoscine butylbromide memiliki efek spasmolitik pada otot polos traktus gastrointestinal, biliaris, urinarius, dan uterus Obat ini umum digunakan untuk mengobati kram perut, kolik ginjal, dan kejang kandung kemih Obat ini mempunyai efek samping yang umumnya seperti kantuk, memicu glaukoma, dan reaksi alergi"
5,1469,Lanareuma 20 mg 10 Kapsul,0.441771,"Lanareuma adalah produk obat dengan zat aktif Piroxicam yang memiliki sifat analgesik (meredakan nyeri), anti-inflamasi (meredakan peradangan) dan antipiretik (menurunkan demam) Lanareuma bekerja dengan cara menghambat reversibel enzim siklooksigenase-1 dan -2 (COX-1 dan -2), sehingga mengakibatkan penurunan pembentukan prekursor prostaglandin"
6,1419,Pronalges Suppositoria 100 mg,0.441397,"PRONALGES SUPPOSITORIA 100 MG mengandung zat aktif Ketoprofen, jenis obat anti inflamasi non steroid (OAINS) Obat ini digunakan untuk mengatasi nyeri ringan hingga sedang pada penyakit arthritis rheumatoid, osteoarthritis, asam urat (gout), ankylosing spondilitis, migrain, sakit kepala, sakit telinga, nyeri pasca operasi, nyeri haid (dismenore) serta nyeri akibat trauma otot Obat ini digunakan dengan cara dimasukkan ke dalam anus"
7,2579,Ottogenta 0.1% Cream 5 g,0.439941,OTTOGENTA KRIM merupakan obat dengan kandungan Gentamicin sulphate dalam bentuk krim Obat ini digunakan untuk infeksi kulit ringan primer maupun sekunder
8,1124,Trifed 10 Tablet,0.432789,"TRIFED merupakan obat yang digunakan untuk mengobati gejala flu (hidung gatal, hidung tersumbat dan bersin-bersin), batuk pilek (selesma) dan allergic rhinitis Trifed mengandung Pseudoephedrine dan Triprolidine Pseudoephedrine adalah zat dekongestan (simpatomimetik) Pseudoephedrine bekerja dengan mengecilkan pembuluh darah untuk mengurangi pembengkakan dan penyumbatan Triprolidine bekerja dengan cara menghambat efek dari zat histamin pada sel-sel tubuh"
9,1090,Komix OBH Kid Sirup Rasa Madu

Query: obat stres


,doc_id,nama_obat,cosine_score,uses
0,2579,Ottogenta 0.1% Cream 5 g,0.622171,OTTOGENTA KRIM merupakan obat dengan kandungan Gentamicin sulphate dalam bentuk krim Obat ini digunakan untuk infeksi kulit ringan primer maupun sekunder
1,948,Crofed 10 Tablet,0.595491,"CROFED merupakan obat yang di gunakan untuk mengobati gejala flu dan allergic rhinitis Crofed mengandung tripolidine dan pesudoefedrin Tripolidine adalah obat golongan antihistamin yang memiliki sifat antikolinergik Obat ini digunakan untuk mengurangi gejala yang terkait dengan alergi Obat ini sering dikombinasikan dengan obat lain untuk mengurangi gejala flu Pesudoefedrin adalah obat yang digunakan sebagai nasal dekongestan, stimulan, dan sebagai wakefulness promoting agent Obat ini termasuk obat simpatomimetik dari kelas phenethylamine dan amfetamin Obat ini biasanya digunakan dalam bentuk garamnya yaitu pseudoephedrine hydrochloride"
2,1584,Paracetamol Enema 125 mg/2.5 ml,0.570088,"Paracetamol Enema adalah obat yang di gunakan sebagai penurun demam dan pereda nyeri seperti sakit kepala, sakit gigi dan nyeri ringan Obat ini dikemas dalam bentuk enema yaitu prosedur pemasukan cairan obat ke dalam kolon melalui anus, untuk anak atau pasien yang kesulitan untuk menelan obat Paracetamol bekerja pada pusat pengatur suhu di hipotalamus untuk menurunkan suhu tubuh (antipiretik) serta menghambat sintesis prostaglandin sehingga dapat mengurangi nyeri ringan sampai sedang (analgesik)"
3,2533,Sulfadiazine Silver Cream 35 g,0.568880,Silver sulfadiazine merupakan obat yang digunakan untuk merawat dan memulihkan infeksi luka dari penggunanya dan luka bakar tubuh
4,2688,Isoniazid 300 mg 10 Tablet,0.567581,"ISONIAZID merupakan obat golongan antibiotik yang digunakan untuk menangani gejala tuberkulosis paru ataupun ekstra paru, dimana biasanya regimen Isoniazid termasuk kedalam regimen penanganan Tuberkulosis (TBC) yang terdiri dari Rifampisin, Izoniazid, Pirazinamid, dan Ethambutol Obat ini bekerja dengan dengan menghambat asam mikolat yang merupakan komponen penting dalam sel bakteri"
5,2710,Ethambutol 500 mg 10 Tablet,0.563188,"ETHAMBUTOL 500 MG TABLET adalah obat antibiotik yang digunakan untuk mengobati tuberkulosis (TBC), terutama bila diduga telah terjadi resistensi Obat ini biasanya digunakan secara kombinasi dengan obat TBC lainnya, seperti isoniazid, rifampicin, dan pyrazinamide"
6,2699,Cotrimoxazole 480 mg 10 Tablet,0.560991,"COTRIMOXAZOLE 480 MG TABLET adalah tablet antibiotik kombinasi trimethoprim 80 mg dan sulfamethoxazole 400 mg Obat ini digunakan untuk menangani infeksi yang disebabkan oleh bakteri, seperti bronkitis, otitis media, dan infeksi saluran kemih Selain itu, kotrimoksazol juga dapat digunakan untuk menangani dan mencegah Pneumocystis Carinii Pneumonia (PCP) pada pasien dengan daya tahan tubuh turun, seperti penderita HIV/AIDS Obat ini bekerja dengan menghambat pembentukan DNA dan protein bakteri"
7,464,Aminophylline 200 mg 100 Tablet,0.560112,"Aminofilin termasuk dalam golongan obat xanthine Obat ini merupakan bronkodilator, yaitu obat yang digunakan untuk membantu mengendurkan otot bronkial atau saluran udara, sehingga dapat meningkatkan aliran udara di paru-paru"
8,2741,Dazolin N Ovula,0.557007,DAZOLIN N OVULA adalah obat ovula dengan kandungan Metronidazole dan Nystatin Metronidazole merupakan obat antibiotik yang bersifat bakterisid Obat ini efektif terhadap amoeba dan Trichomonas vaginalis (trikomonasid) Nystatin merupakan obat antifungal yang bersifat fungisida Obat ini efektif terhadap Candida Sp Dengan demikian obat ini digunakan untuk mengobati infeksi pada area vagina yang disebabkan oleh Trichomonas vaginalis dan Candida albicans
9,3034,Dazolin N Ovula,0.557007,DAZOLIN N OVULA adalah obat ovula dengan kandungan Metronidazole dan Nystatin Metronidazole merupakan obat antibiotik yang bersifat bakterisid Obat ini efektif terhadap amoeba dan Trichomonas vaginalis (trikomonasid) Nystatin merupakan obat antifungal yang bersifat fungisida Obat ini efe

Query: krim wajah


...

,doc_id,nama_obat,cosine_score,uses
0,2468,Bioplacenton Gel 15 g,0.446844,"BIOPLACENTON merupakan obat yang mengandung Placenta Extract dan Neomycin sulfate Obat ini biasa digunakan untuk mengobati luka bakar, luka dengan infeksi, serta luka kronik dan jenis luka yang lain Ekstrak plasenta bekerja dengan memicu pembentukan jaringan baru dan untuk wound healing, sedangkan neomycin bekerja dengan mencegah atau mengatasi infeksi bakteri Gram negatif pada area luka"
1,2616,Mebo Salep 20 gr,0.432742,"MEBO SALEP 20 GR merupakan salah satu sediaan salep herbal luka bakar yang diformulasikan dengan bahan-bahan herbal alami seperti : Ekstrak Akar mangkokan, ekstrak kulit chinensis Phellodendri, Ekstrak rimpang Coptidis"
2,2570,Mertus Cream 10 gr,0.418609,"Mertus adalah antibiotik topikal (untuk pemakaian luar) yang digunakan untuk mengobati infeksi primer dan sekunder, seperti impetigo (infeksi kulit yang berupa lepuh atau bercak luka terbuka pada kulit), luka bakar, luka pasca operasi, dan luka terinfeksi Mertus mengandung zat aktif Mupirocin calcium bersifat bakteriostatik (menghambat pertumbuhan bakteri) dan bakterisidal (membunuh bakteri) pada konsentrasi yang lebih tinggi"
3,2603,Lanakeloid-E Cream 10 g,0.416667,"LANAKELOID-E CREAM merupakan obat dengan kandungan Centella Asiatica Phytosome, Vit E dalma bentuk krim Obat ini digunakan untuk membantu proses penyembuhan luka bakar ringan"
4,2480,Burnazin 10 mg/g Cream 35 g,0.403786,BURNAZIN CREAM adalah obat yang mengandung Silver Sulfadiazine Obat ini digunakan untuk mengobati atau mencegah infeksi pada luka bakar yang rentan terhadap infeksi sekunder
5,2632,Lanakeloid 10 Kaplet,0.396526,LANAKELOID merupakan obat dengan kandungan Centella Asiatica dalam bentuk kaplet Obat ini digunakan untuk membantu meringankan luka bakar ringan
6,2009,Faktu Salep,0.350823,"FAKTU SALEP adalah obat yang digunakan untuk mengobati ambeien/wasir internal dan eksternal yang terkait dengan gejala inflamasi dan perdarahan Faktu salep mengandung policresulen 50 mg dan cinchocaine 10 mg Policresulen adalah obat yang digunakan sebagai hemostatik topikal dan antiseptik Policresulen adalah produk kondensasi metacresolsulfonic acid dan methanal Policresulen bekerja dengan membuat sedikit luka bakar kimiawi di daerah yang luka, zat ini mampu membunuh bakteri sehingga menghentikan terjadinya proses infeksi lanjutan Cinchocaine adalah anestesi lokal golongan amida, Senyawa ini adalah salah satu anestesi paling ampuh dari anestesi lokal aksi panjang"
7,2533,Sulfadiazine Silver Cream 35 g,0.348367,Silver sulfadiazine merupakan obat yang digunakan untuk merawat dan memulihkan infeksi luka dari penggunanya dan luka bakar tubuh
8,2540,Centabio Gel 20 g,0.324938,"CENTABIO GEL 20 G adalah obat yang digunakan untuk membantu penyembuhan luka bakar, borok kronis, dan cacat kulit yang disertai infeksi bakteri tertentu Centabio mengandung neomycin yang bekerja dengan cara mengikat secara reversibel terhadap sub unit 30s dari ribosom bakteri sehingga menghambat sintesa protein yang pada akhirnya menghambat pertumbuhan bakteri itu"
9,2577,Neocenta Gel 15 g,0.296756,"Neocenta mengandung Neomycin Sulfate dan Placenta Ekstrak yang diindikasikan untuk luka bakar, ulkus kronis & dekubital, eksim (peradangan kulit), pioderma (infeksi kulit yang di sebabkan bakteri), impetigo"


Query: obat herbal apa yang sudah terdaftar BPOM


,doc_id,nama_obat,cosine_score,uses
0,1235,Komix Herbal 15 ml 4 Botol,0.372104,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
1,1596,Komix Herbal 15 ml 4 Botol,0.372104,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
2,1246,Komix Herbal Lemon Sirup 15 ml 4 Botol,0.370211,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
3,1607,Komix Herbal Lemon Sirup 15 ml 4 Botol,0.370211,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
4,1247,Komix Herbal Original 15 ml 6 Sachet,0.368345,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
5,1277,Komix Herbal Rasa Jeruk Nipis 15 ml 6 Sachet,0.368345,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
6,1608,Komix Herbal Original 15 ml 6 Sachet,0.368345,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
7,1638,Komix Herbal Rasa Jeruk Nipis 15 ml 6 Sachet,0.368345,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
8,1278,Komix Herbal Rasa Peppermint 15 ml 6 Sachet,0.366508,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"
9,1639,Komix Herbal Rasa Peppermint 15 ml 6 Sachet,0.366508,"KOMIX HERBAL merupakan obat herbal yang mengandung bahan alami berupa esktrak lagundi, ekstrak jahe merah, thymi herba, licorice, peppermint oil dan madu Obat herbal ini digunakan untuk membantu meredakan batuk berdahak"


Query: obat flu untuk balita


,doc_id,nama_obat,cosine_score,uses
0,948,Crofed 10 Tablet,0.547399,"CROFED merupakan obat yang di gunakan untuk mengobati gejala flu dan allergic rhinitis Crofed mengandung tripolidine dan pesudoefedrin Tripolidine adalah obat golongan antihistamin yang memiliki sifat antikolinergik Obat ini digunakan untuk mengurangi gejala yang terkait dengan alergi Obat ini sering dikombinasikan dengan obat lain untuk mengurangi gejala flu Pesudoefedrin adalah obat yang digunakan sebagai nasal dekongestan, stimulan, dan sebagai wakefulness promoting agent Obat ini termasuk obat simpatomimetik dari kelas phenethylamine dan amfetamin Obat ini biasanya digunakan dalam bentuk garamnya yaitu pseudoephedrine hydrochloride"
1,1020,Longatin 25 mg 4 Kapsul,0.453743,LONGATIN 25 MG KAPSUL merupakan obat yang mengandung zat aktif Noscapin Obat ini digunakan untuk membantu mengatasi batuk kering
2,2579,Ottogenta 0.1% Cream 5 g,0.439941,OTTOGENTA KRIM merupakan obat dengan kandungan Gentamicin sulphate dalam bentuk krim Obat ini digunakan untuk infeksi kulit ringan primer maupun sekunder
3,877,Tremenza 10 Tablet,0.429064,TREMENZA merupakan obat dengan kandungan Pseudoephedrine HCl dan Triprolidine HCl Obat ini digunakan untuk meringankan gejala-gejala flu Pseudoefedrin dalam obat ini bekerja pada reseptor alfa-adrenergik dalam mukosa saluran pernafasan sehingga menghasilkan vasokonstriksi Triprolidin dalam obat ini merupakan suatu antihistamin yang bekerja sebagai antagonis reseptor histamin H1 dalam pengobatan alergi
4,904,Tuzalos 4 Kaplet,0.415008,"TUZALOS merupakan obat batuk dan pereda flu dengan kandungan Paracetamol, Phenylpropanolamine, Dextromethorphan HBr, dan Chlorpheniramine maleat Dextromethorphan HBr sebagai antitusif yang bisa menekan refleks batuk Paracetamol digunakan sebagai pereda demam dan sakit kepala, Phenylpropanolamine digunakan untuk mengobati gejala hidung tersumbat Chlorpheniramine maleate bekerja sebagai antihistamin atau anti alergi sehingga obat ini digunakan untuk mengatasi gejala flu seperti demam, sakit kepala, hidung tersumbat, dan bersin-bersin yang disertai batuk"
5,1584,Paracetamol Enema 125 mg/2.5 ml,0.403113,"Paracetamol Enema adalah obat yang di gunakan sebagai penurun demam dan pereda nyeri seperti sakit kepala, sakit gigi dan nyeri ringan Obat ini dikemas dalam bentuk enema yaitu prosedur pemasukan cairan obat ke dalam kolon melalui anus, untuk anak atau pasien yang kesulitan untuk menelan obat Paracetamol bekerja pada pusat pengatur suhu di hipotalamus untuk menurunkan suhu tubuh (antipiretik) serta menghambat sintesis prostaglandin sehingga dapat mengurangi nyeri ringan sampai sedang (analgesik)"
6,2533,Sulfadiazine Silver Cream 35 g,0.402259,Silver sulfadiazine merupakan obat yang digunakan untuk merawat dan memulihkan infeksi luka dari penggunanya dan luka bakar tubuh
7,2688,Isoniazid 300 mg 10 Tablet,0.401340,"ISONIAZID merupakan obat golongan antibiotik yang digunakan untuk menangani gejala tuberkulosis paru ataupun ekstra paru, dimana biasanya regimen Isoniazid termasuk kedalam regimen penanganan Tuberkulosis (TBC) yang terdiri dari Rifampisin, Izoniazid, Pirazinamid, dan Ethambutol Obat ini bekerja dengan dengan menghambat asam mikolat yang merupakan komponen penting dalam sel bakteri"
8,893,Decolgen 4 Tablet,0.400892,"DECOLGEN merupakan obat flu dengan kandungan Paracetamol, Phenylpropanolamine HCl, dan Chlorpheniramine maleate Paracetamol digunakan sebagai pereda demam dan sakit kepala Phenylpropanolamine digunakan untuk mengobati gejala hidung tersumbat Chlorpheniramine maleate bekerja sebagai antihistamin atau anti alergi sehingga obat ini digunakan untuk meredakan gejala flu seperti sakit kepala, demam, bersin-bersin dan hidung tersumbat"
9,2710,Ethambutol 500 mg 10 Tablet,0.398234,"ETHAMBUTOL 500 MG TABLET adalah obat antibiotik yang digunakan untuk mengobati tuberkulosis (TBC), terutama bila diduga telah terjadi resistensi Obat ini biasanya digunakan secara kombinasi dengan obat TBC lainnya, sepert